In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import genjax
genjax.pretty()

In [ ]:
## Load a video ##

import condorgmm
import condorgmm.data as data

video = (
    data.YCBVVideo.training_scene(2)
    .downscale(2)
)

import matplotlib.pyplot as plt
plt.imshow(video[1].rgb)

In [ ]:
import rerun as rr

In [ ]:
import condorgmm.condor.interface.camera_tracking as ct

frame=video[0]
n_gaussians=300
camera_pose_world_frame=condorgmm.Pose(video[0].camera_pose)

# takes <3 seconds to run after jitting
gmm, ccts, hypers = ct.initialize(frame, n_gaussians, camera_pose_world_frame, get_hypers=True)

In [ ]:
import jax
from condorgmm.condor.model.model import step_model

@jax.jit
def do_sim(state, hypers):
    tr = step_model.simulate(
        jax.random.key(0),
        (
            state,
            hypers
        )
    )
    return tr.get_retval()

In [ ]:
import jax.numpy as jnp

def _f(x):
    return jnp.array(x, dtype=jnp.float32)

st2 = do_sim(
    ccts.state.replace({"scene": {"background_rigidity": jnp.array(.01, dtype=float)}}),
    hypers.replace(
        xyz_cov_evolution_pcnt=jnp.array(20., dtype=float),
        alpha_multiplier_for_evolved_gaussian=jnp.array(40., dtype=float),
        target_rgb_mean_evolution_variance=_f(10.0) ** 2,
    )
)

In [ ]:
from condorgmm.condor.rerun import log_state
import numpy as np
import rerun as rr
condorgmm.rr_init("condor2/frame0_00")
rr.set_time_sequence("frame", 0)
log_state(ccts.state, hypers)
rr.log("depth_img/observation", rr.DepthImage(np.array(frame.depth)))
rr.log("depth_img/inferred", rr.DepthImage(np.array(ccts.state.datapoints.value.xyz[..., 2].reshape(frame.depth.shape))))

In [ ]:
rr.set_time_sequence("frame", 1)
log_state(st2, hypers)